In [3]:
import requests
import pandas as pd

geoapify_key = "ba85a93e0db14167ada7e83d81a7e7bf"

In [5]:
# List of all U.S. states
states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 
    'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 
    'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 
    'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 
    'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 
    'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 
    'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 
    'Wisconsin', 'Wyoming'
]

state_coords = {}

In [21]:
# To get coordinates for a state
def get_coordinates(state):
    url = f"https://api.geoapify.com/v1/geocode/search?text={state}, United States&apiKey={geoapify_key}"
    response = requests.get(url)
    data = response.json()
    
def get_coordinates(state):
    url = f"https://api.geoapify.com/v1/geocode/search?text={state}, United States&apiKey={geoapify_key}"
    response = requests.get(url)
    data = response.json()

    if response.status_code == 200 and data['features']:
        
        coords = data['features'][0]['geometry']['coordinates']
        return coords[1], coords[0]  
    else:
        print(f"Coordinates for {state} not found.")
        return None, None

In [25]:
# Loop through states and get coordinates
for state in states:
    lat, lon = get_coordinates(state)
    if lat and lon:
        state_coords[state] = [lat, lon]
        print(f"{state}: {lat}, {lon}")

Alabama: 33.2588817, -86.8295337
Alaska: 64.4459613, -149.680909
Arizona: 34.395342, -111.763275
Arkansas: 35.2048883, -92.4479108
California: 36.7014631, -118.755997
Colorado: 38.7251776, -105.607716
Connecticut: 41.6500201, -72.7342163
Delaware: 38.6920451, -75.4013315
Florida: 27.7567667, -81.4639835
Georgia: 32.3293809, -83.1137366
Hawaii: 19.593801499999998, -155.42837009716908
Idaho: 43.6447642, -114.015407
Illinois: 40.0796606, -89.4337288
Indiana: 40.3270127, -86.1746933
Iowa: 41.9216734, -93.3122705
Kansas: 38.27312, -98.5821872
Kentucky: 37.5726028, -85.1551411
Louisiana: 30.8703881, -92.007126
Maine: 45.709097, -68.8590201
Maryland: 39.5162401, -76.9382069
Massachusetts: 42.3788774, -72.032366
Michigan: 43.6211955, -84.6824346
Minnesota: 45.9896587, -94.6113288
Mississippi: 32.9715285, -89.7348497
Missouri: 38.7604815, -92.5617875
Montana: 47.3752671, -109.638757
Nebraska: 41.7370229, -99.5873816
Nevada: 39.5158825, -116.853722
New Hampshire: 43.4849133, -71.6553992
New Jers

In [27]:
# Convert to DataFrame for easier manipulation
state_coords_df = pd.DataFrame.from_dict(state_coords, orient='index', columns=['Latitude', 'Longitude'])
state_coords_df = state_coords_df.reset_index().rename(columns={'index': 'State'})

In [29]:
# Saved coordinates to a CSV file
state_coords_df.to_csv('/Users/latifahjones/Desktop/Project-3/Sorted_Tables/state_coordinates.csv', index=False)

# Display the DataFrame
state_coords_df.head()

,State,Latitude,Longitude
0,Alabama,33.258882,-86.829534
1,Alaska,64.445961,-149.680909
2,Arizona,34.395342,-111.763275
3,Arkansas,35.204888,-92.447911
4,California,36.701463,-118.755997


In [33]:
import pandas as pd
#Loading state coordinates 
coords_df = pd.read_csv('/Users/latifahjones/Desktop/Project-3/Sorted_Tables/state_coordinates.csv')

# Loading state industry demand data
demand_df = pd.read_csv('/Users/latifahjones/Desktop/Project-3/Sorted_Tables/Industry_Demand_per_state.csv')

In [37]:
# Finding the top 2 industries for each state
top_industries = demand_df.set_index('State').apply(lambda x: x.nlargest(2).index.tolist(), axis=1)

# Creating a new DataFrame with states and their top 2 industries
top_industries_df = pd.DataFrame(top_industries, columns=['Top_Industries'])
top_industries_df = top_industries_df.reset_index()

# Merge with coordinates data
map_data = pd.merge(coords_df, top_industries_df, on='State')
map_data.head()


,State,Latitude,Longitude,Top_Industries
0,Alabama,33.258882,-86.829534,"[Total nonfarm, Trade, transportation, and uti..."
1,Alaska,64.445961,-149.680909,"[Total nonfarm, Government]"
2,Arizona,34.395342,-111.763275,"[Total nonfarm, Trade, transportation, and uti..."
3,Arkansas,35.204888,-92.447911,"[Total nonfarm, Trade, transportation, and uti..."
4,California,36.701463,-118.755997,"[Total nonfarm, Education and health services]"


In [39]:
import folium

# Base map centered on the United States
m = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Adding markers for each state with the top 2 industries in demand
for idx, row in map_data.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['State']}: Top Industries - {', '.join(row['Top_Industries'])}",
        icon=folium.Icon(color="blue")
    ).add_to(m)

# Display the map
m

In [41]:
import pandas as pd

# Load the CSV file
file_path = '/Users/latifahjones/Desktop/Project-3/Sorted_Tables/Industry_Demand_per_state.csv'
df = pd.read_csv(file_path)

# Display the first few rows to inspect the data
df.head()


,State,Total nonfarm,Mining and logging,"Mining, logging, and construction",Construction,Manufacturing,"Trade, transportation, and utilities",Information,Financial activities,Professional and business services,Education and health services,Leisure and hospitality,Other services,Government
0,Alabama,2217.1,8.9,116.7,107.8,289.6,417.5,24.2,105.0,264.8,271.9,210.9,104.9,411.6
1,Alaska,339.9,12.0,33.2,21.2,13.9,67.5,4.2,11.1,29.1,54.0,36.8,11.8,78.3
2,Arizona,3271.5,15.5,233.7,218.2,191.8,629.8,50.0,243.2,477.7,556.4,348.8,104.6,435.5
3,Arkansas,1372.5,5.3,71.4,66.1,162.7,271.0,12.3,69.6,158.6,217.9,129.0,67.1,212.9
4,California,18097.7,19.6,941.9,922.3,1298.5,3123.1,519.2,810.3,2788.0,3287.4,2041.6,602.0,2685.7


In [43]:
# Specify the columns to remove
columns_to_remove = ['Total nonfarm', 'Mining, logging, and construction']

# Drop the columns from the DataFrame
df = df.drop(columns=columns_to_remove)

# Verify that the columns have been removed
df.head()


,State,Mining and logging,Construction,Manufacturing,"Trade, transportation, and utilities",Information,Financial activities,Professional and business services,Education and health services,Leisure and hospitality,Other services,Government
0,Alabama,8.9,107.8,289.6,417.5,24.2,105.0,264.8,271.9,210.9,104.9,411.6
1,Alaska,12.0,21.2,13.9,67.5,4.2,11.1,29.1,54.0,36.8,11.8,78.3
2,Arizona,15.5,218.2,191.8,629.8,50.0,243.2,477.7,556.4,348.8,104.6,435.5
3,Arkansas,5.3,66.1,162.7,271.0,12.3,69.6,158.6,217.9,129.0,67.1,212.9
4,California,19.6,922.3,1298.5,3123.1,519.2,810.3,2788.0,3287.4,2041.6,602.0,2685.7


In [45]:
# Save the cleaned DataFrame to a new CSV file
output_path = '/Users/latifahjones/Desktop/Project-3/Sorted_Tables/Industry_Demand_per_state_cleaned.csv'
df.to_csv(output_path, index=False)

print(f"Updated data saved to {output_path}")

Updated data saved to /Users/latifahjones/Desktop/Project-3/Sorted_Tables/Industry_Demand_per_state_cleaned.csv
